In [155]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.layers.experimental import preprocessing
import os
import cv2
import glob

In [156]:
def logoOverlay(image,logo,alpha=1.0,x=0, y=0, scale=1.0):
    (h, w) = image.shape[:2]
    image = np.dstack([image, np.ones((h, w), dtype="uint8") * 255])

    overlay = cv2.resize(logo, None,fx=scale,fy=scale)
    (wH, wW) = overlay.shape[:2]
    output = image.copy()
    # blend the two images together using transparent overlays
    try:
        if x<0 : x = w+x
        if y<0 : y = h+y
        if x+wW > w: wW = w-x  
        if y+wH > h: wH = h-y
        print(x,y,wW,wH)
        overlay=cv2.addWeighted(output[y:y+wH, x:x+wW],alpha,overlay[:wH,:wW],1.0,0)
        output[y:y+wH, x:x+wW ] = overlay
    except Exception as e:
        print("Error: Logo position is overshooting image!")
        print(e)

    output= output[:,:,:3]
    return output

In [157]:
CURRENT_PATH = os.getcwd()
CURRENT_PATH = "\\".join(CURRENT_PATH.split('\\')[:-2])
PNG_PATH = "\\images\\bg_remove\\"
BG_PATH = "\\images\\bg\\"
SAVE_PATH = "\\images\\result\\"

In [158]:
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.2,
    zoom_range=[0.5,1.5],
    fill_mode='nearest',
    brightness_range=[-0.5, 1.5],
    channel_shift_range=[]
)

In [159]:
for i, bg_name in enumerate(glob.glob(CURRENT_PATH + BG_PATH + "*jpg")):
    bg_img = Image.open(bg_name)
    for j, fg_name in enumerate(glob.glob(CURRENT_PATH + PNG_PATH + "*png")):
        fg_img = Image.open(fg_name)
        fg_img = np.asarray(fg_img).reshape(1,600,600,4)
        for k in range(10):
            bg = Image.fromarray(np.copy(np.asarray(bg_img)))
            img = next(datagen.flow(fg_img))[0]
            img = Image.fromarray(img.astype('uint8'), 'RGBA')
            bg.paste(img, (0, 0), img)
            n = fg_name.split('\\')[-1].split('.')[0]
            bg.save(CURRENT_PATH + SAVE_PATH + f"BG{i}_FG{j}_AUG{k}.jpg")

KeyboardInterrupt: 